#Task list:
-Once this first basic step has been done the next step is to get this configure with the old model (fully dressed quarks). 
-then learn how to make cuts and limitations on the data.
-then learn how to make a data generator by changing various things in the generating function to be representative in the new model.

In [ ]:
#Spring model
import numpy as np
def dydt(t, y, v, k, m,RL):
    #v[0:12] =[xd1 , yd1 , zd1 , xd2 , yd2 , zd2 , xd3 , yd3 , zd3 , xdc , ydc , zdc]
    return v

def dvdt(t, y, v, k, m,RL):
    #y[0:12]    =[x1 , y1 , z1 , x2 , y2 , z2 , x3 , y3 , z3 , xc , yc , zc]
    #Imposing limitation(this is the hard part):
    y[y>1000]=1000
    
    dvdt = np.empty_like(y)
    m1 , m2 , m3 , Mc = m[0] , m[1] , m[2] , m[3]
    pos1 , pos2, pos3 , posc= y[0:3] , y[3:6] , y[6:9] , y[9:12]
    
    dvdt[0:3]= -k*(y[9:12]-y[0:3])/m1 * ( -1  +RL/(np.sum((posc-pos1)**2)**0.5)  )
    dvdt[3:6]= -k*(y[9:12]-y[3:6])/m2 * ( -1  +RL/(np.sum((posc-pos2)**2)**0.5)  )
    dvdt[6:9]= -k*(y[9:12]-y[6:9])/m3 * ( -1  +RL/(np.sum((posc-pos3)**2)**0.5) )
    #dvdt[9:12]=0
    for j in range(0,3):
        dvdt[j+9]=  k*(y[j+9]-y[j])/Mc  *  (-1 + 2*RL/( np.sum((posc-pos1)**2)**0.5)  ) \
                    +k*(y[j+9]-y[j+3])/Mc* (-1 + 2*RL/( np.sum((posc-pos2)**2)**0.5)  ) \
                    +k*(y[j+9]-y[j+6])/Mc* (-1 + 2*RL/( np.sum((posc-pos3)**2)**0.5)  )
    
    #print( np.sum((posc-pos1)**2)**0.5 , np.sum((posc-pos2)**2)**0.5 ,np.sum((posc-pos3)**2)**0.5) 
    return dvdt

def runge_kutta(t0, y0, v0, t_end, dt, k, m,RL):
    t_values = np.arange(t0, t_end, dt)
    y_values = np.empty((len(t_values),12))
    v_values = np.empty((len(t_values),12))

    y_values[0] = y0
    v_values[0] = v0

    for i in range(1, len(t_values)):
        t = t_values[i-1]
        y = y_values[i-1]
        v = v_values[i-1]
        
        k1_y = dydt(t, y, v, k, m,RL)
        k1_v = dvdt(t, y, v, k, m,RL)

        k2_y = dydt(t + dt/2, y + k1_y*dt/2, v + k1_v*dt/2, k, m,RL)
        k2_v = dvdt(t + dt/2, y + k1_y*dt/2, v + k1_v*dt/2, k, m,RL)

        k3_y = dydt(t + dt/2, y + k2_y*dt/2, v + k2_v*dt/2, k, m,RL)
        k3_v = dvdt(t + dt/2, y + k2_y*dt/2, v + k2_v*dt/2, k, m,RL)

        k4_y = dydt(t + dt, y + k3_y*dt, v + k3_v*dt, k, m,RL)
        k4_v = dvdt(t + dt, y + k3_y*dt, v + k3_v*dt, k, m,RL)

        y_values[i] = y + (dt/6) * (k1_y + 2*k2_y + 2*k3_y + k4_y)
        v_values[i] = v + (dt/6) * (k1_v + 2*k2_v + 2*k3_v + k4_v)

    return t_values, y_values, v_values

t0 = 0     # Initial time
t_end = 75 # End time
dt = 0.001  # Time step
k = 20.91    # Constant k
m = np.array([2,1,1,1e-5])    # Constant m
RL = 0

x1 , y1 , z1= float(5.0),  float(0.0 ), float(0.0)
x2 , y2 , z2= float(-2.5), float(4.330127) , float(0.0)
x3 , y3 , z3= float(-2.5), float(-4.330127) , float(0.0)
xc , yc , zc= float(0.0) , float(0.0) , float(0.0)
                                           
xd1 , yd1 , zd1= float(5.0) , float(-5.0) , float(0.0)
xd2 , yd2 , zd2= float(0.0), float(10.0) , float(-10.0)
xd3 , yd3 , zd3= float(-10.0) , float(0.0) , float(10.0)
xdc , ydc , zdc= float(0.0) , float(0.0) , float(0.0)

y0 , v0 = np.empty(12) , np.empty(12)
y0[0:12]    =[x1 , y1 , z1 , x2 , y2 , z2 , x3 , y3 , z3 , xc , yc , zc]
v0[0:12]    =[xd1 , yd1 , zd1 , xd2 , yd2 , zd2 , xd3 , yd3 , zd3 , xdc , ydc , zdc]
#print( dvdt(t0, y0, v0, k, m,RL))
t_values, y_values, v_values = runge_kutta(t0, y0, v0, t_end, dt, k, m,RL)

np.savetxt('motion_data.txt', np.column_stack((t_values, y_values[:,0],y_values[:,1],
                                                  y_values[:,2] , y_values[:,3],
                                                  y_values[:,4], y_values[:,5], 
                                                  y_values[:,6]  , y_values[:,7],
                                                  y_values[:,8], y_values[:,9],
                                                  y_values[:,10], y_values[:,11]  ) 
                                                ) , delimiter=',' , 
header='Time (s), x1 , y1 , z1 , x2 , y2 , z2 , x3 , y3 , z3 , xc , yc , zc ')

In [ ]:
#Spring+EM+Magnetic Field Model
import numpy as np
μ0=4*np.pi*1e-7
Q    =  1.602e-19
k_EM =  9e9
B_mag=  10

def dydt(t, y, v, a, m ,q , k_SP , RL, EM=True):
    #v[0:12] =[xd1 , yd1 , zd1 , xd2 , yd2 , zd2 , xd3 , yd3 , zd3 , xdc , ydc , zdc]
    return v

def dvdt(t, y, v, a, m ,q , k_SP , RL, EM=True):
    #y[0:12]    =[x1 , y1 , z1 , x2 , y2 , z2 , x3 , y3 , z3 , xc , yc , zc]
    #Imposing limitation(this is the hard part):
    #y[y>1000]=1000

    
    Y=np.array([y[0:3] , y[3:6] , y[6:9] , y[9:12]])
    V=np.array([v[0:3] , v[3:6] , v[6:9] , v[9:12]])
    dVdt_in=np.array([a[0:3] , a[3:6] , a[6:9] , a[9:12]])
    dVdt_fi=np.empty_like(dVdt_in)
    #Spring force for center
    for j in range(0,3):
        dVdt_in[3,j]=   k_SP*(Y[3,j]-Y[0,j])/m[3]* (-1 + 2*RL/( np.sum((Y[3]-Y[0])**2)**0.5)  ) \
                    +k_SP*(Y[3,j]-Y[1,j])/m[3]* (-1 + 2*RL/( np.sum((Y[3]-Y[1])**2)**0.5)  ) \
                    +k_SP*(Y[3,j]-Y[2,j])/m[3]* (-1 + 2*RL/( np.sum((Y[3]-Y[2])**2)**0.5)  )
        
    
    for i in range(0,3):
        #Spring Force
        A= -k_SP * (Y[3]-Y[i]) * ( -1  +RL/(np.sum((Y[3]-Y[i])**2)**0.5)  )
        #Magnetic Force
        vi=(np.sum(V[i]**2))**0.5
        velaccel_i = np.array([ V[i,1]*dVdt_in[i,1]+V[i,2]*dVdt_in[i,2] ,\
                                V[i,0]*dVdt_in[i,0]+V[i,2]*dVdt_in[i,2] ,\
                                V[i,0]*dVdt_in[i,0]+V[i,1]*dVdt_in[i,1] ,])
        
        F= m[i] -  B_mag*q[i]*(vi**-1 - V[i]**2 * (vi)**-3)
        C= -V[i]*(B_mag*q[i]) *velaccel_i*vi**(-3) # need fixing
        #EM Force
        B,D,E= np.zeros(3),np.zeros(3) , np.zeros(3)
        
        for k in range(0,3):            
            if(k==i):
                continue
            vk = (np.sum(V[k]**2))**0.5
            const=0.5*q[i]*q[k]
            velaccel_k = np.array([ V[k,1]*dVdt_in[k,1]+V[k,2]*dVdt_in[k,2] ,\
                                    V[k,0]*dVdt_in[k,0]+V[k,2]*dVdt_in[k,2] ,\
                                    V[k,0]*dVdt_in[k,0]+V[k,1]*dVdt_in[k,1]  ])
            #F += const*(1e-7)/(np.sum(np.abs(Y[i]-Y[k])**2)**0.5) * (vk*vi**-1 - V[i]**2 *vk*vi**-3)
            print()
            F+= (vi**-1 - V[i]**2 * (vi)**-3)*const*(1e-7)*vk/(np.sum(np.abs(Y[i]-Y[k])**2)**0.5)
            B +=0.5*q[i]*q[k]*(  1e-7*vi*vk - k_EM )/(np.sum(np.abs(Y[i]-Y[k])**2)**1.5) * (-(Y[i]-Y[k]))
            
            D += -V[i]*const*(1e-7)*(np.sum(V[k]*dVdt_in[k])/(vi*vk) - (velaccel_i)*vk*vi**-3)/(np.sum(np.abs(Y[i]-Y[k])**2)**0.5)
            E += V[i]*const*(1e-7) * (np.sum((Y[i]-Y[k])*(V[i]-V[k]))/(np.sum(np.abs(Y[i]-Y[k])**2)**1.5) ) * vk/vi

        dVdt_fi[i] =(A+B+C+D+E)/F
        
        print('A, ' ,A)
        print('B, ' ,B)
        print('-V[i]*(B_mag*q[i]) ' , -V[i]*(B_mag*q[i]))
        print('velaccel_i[i]*vi**(-3) ', velaccel_i*vi**(-3))
        print('C, ' ,C)
        
        print('D, ' ,D)
        print('E, ' ,E)
        print('F, ' ,F)
        #print()

    print()
    print()
    dvdt=dVdt_fi.flatten()    
    #print( np.sum((posc-pos1)**2)**0.5 , np.sum((posc-pos2)**2)**0.5 ,np.sum((posc-pos3)**2)**0.5) 
    return dvdt
def runge_kutta(t0,t_end, dt, y, v, a, m ,q , k_SP , RL, EM):
    t_values = np.arange(t0, t_end, dt)
    y_values = np.empty((len(t_values),12))
    v_values = np.empty((len(t_values),12))
    a_values = np.empty((len(t_values),12))

    y_values[0] = y
    v_values[0] = v
    a_values[0] = a

    for i in range(1, len(t_values)):
    #for i in range(1, len(t_values)):
        t = t_values[i-1]
        y = y_values[i-1]
        v = v_values[i-1]
        a = a_values[i-1]

        print('Loop number ', i)
        a_values[i] = dvdt(t, y, v, a,  m ,q , k_SP , RL, EM)
        print('a_values goes here' , a_values[i])
        print()
        print()
        print()
        
        k1_y = dydt(t, y, v, a,  m ,q , k_SP , RL, EM)
        k1_v = dvdt(t, y, v, a,  m ,q , k_SP , RL, EM)

        k2_y = dydt(t + dt/2, y + k1_y*dt/2, v + k1_v*dt/2,  a, m ,q , k_SP , RL, EM)
        k2_v = dvdt(t + dt/2, y + k1_y*dt/2, v + k1_v*dt/2,  a, m ,q , k_SP , RL, EM)

        k3_y = dydt(t + dt/2, y + k2_y*dt/2, v + k2_v*dt/2,  a, m ,q , k_SP , RL, EM)
        k3_v = dvdt(t + dt/2, y + k2_y*dt/2, v + k2_v*dt/2,  a, m ,q , k_SP , RL, EM)

        k4_y = dydt(t + dt, y + k3_y*dt, v + k3_v*dt, a, m ,q , k_SP , RL, EM)
        k4_v = dvdt(t + dt, y + k3_y*dt, v + k3_v*dt, a, m ,q , k_SP , RL, EM)

        y_values[i] = y + (dt/6) * (k1_y + 2*k2_y + 2*k3_y + k4_y)
        v_values[i] = v + (dt/6) * (k1_v + 2*k2_v + 2*k3_v + k4_v)
        print('End of Loop number ',i)
        print(y_values[i])
        print(v_values[i])
        print(a_values[i])
        print()
        print()
        print()
    return t_values, y_values, v_values
t0 = 0     # Initial time
t_end = 3 # End time
dt = 1.0  # Time step
k_SP = 20.91    # Spring Constant k
m = np.array([3.56e-30,3.56e-30,3.56e-30,1e-45])    # Constant m
q = np.array([Q/3, -Q/3,  Q/3])
RL = 1e-15

x1 , y1 , z1= float(5.0e-15 ),  float(0.0e-15  ), float(0.0e-15 )
x2 , y2 , z2= float(-2.5e-15 ), float(4.330127e-15 ) , float(0.0e-15 )
x3 , y3 , z3= float(-2.5e-15 ), float(-4.330127e-15 ) , float(0.0e-15 )
xc , yc , zc= float(0.0e-15 ) , float(0.0e-15 ) , float(0.0e-15 )
                                 
xd1 , yd1 , zd1= float(5.0e-15 )  , float(-5.0e-15 ) , float(0.0e-15 )
xd2 , yd2 , zd2= float(0.0e-15 )  , float(5.0e-15 )  , float(-5.0e-15 )
xd3 , yd3 , zd3= float(-5.0e-15 ) , float(0.0e-15 )  , float(5.0e-15 )
xdc , ydc , zdc= float(0.0e-15 )  , float(0.0e-15 )  , float(0.0e-15 )

xdd1 , ydd1 , zdd1= float(-1.0e-15 )  , float(-2.0e-15 ) , float(-3.0e-15 )
xdd2 , ydd2 , zdd2= float(-4.0e-15 )  , float(-5.0e-15 ) , float(-6.0e-15 )
xdd3 , ydd3 , zdd3= float(-7.0e-15 )  , float(-8.0e-15 ) , float(-9.0e-15 )
xddc , yddc , zddc= float(0.0e-15 )  , float(0.0e-15 ) , float(0.0e-15 )

y0 , v0 ,a0= np.empty(12) , np.empty(12) , np.empty(12)
y0[0:12]    =[x1 , y1 , z1 , x2 , y2 , z2 , x3 , y3 , z3 , xc , yc , zc]
v0[0:12]    =[xd1 , yd1 , zd1 , xd2 , yd2 , zd2 , xd3 , yd3 , zd3 , xdc , ydc , zdc]
a0[0:12]    =[xdd1 , ydd1 , zdd1,  xdd2 , ydd2 , zdd2, xdd3 , ydd3 , zdd3 , xdc , ydc , zdc]
EM=True   
#print( dvdt(t0, y0, v0, a, m ,q , k_SP , RL, EM) )
    

t_values, y_values, v_values = runge_kutta(t0,t_end, dt, y0, v0, a0, m ,q , k_SP , RL, EM)
print()                  
""" 
np.savetxt('motion_datawithEM+mag-ActualScale.txt', np.column_stack((t_values, 
                                                  y_values[:,0],y_values[:,1] , y_values[:,2] ,
                                                  y_values[:,3],y_values[:,4] , y_values[:,5], 
                                                  y_values[:,6],y_values[:,7] , y_values[:,8],
                                                  y_values[:,9],y_values[:,10], y_values[:,11]  ) 
                                                ) , delimiter=',' , 
header='Time (s), x1 , y1 , z1 , x2 , y2 , z2 , x3 , y3 , z3 , xc , yc , zc ')
"""